In [2]:
from datasets import load_dataset
from collections import Counter

# Carica il dataset Sentiment140
dataset = load_dataset("stanfordnlp/sentiment140", trust_remote_code=True)

# Conta le occorrenze delle etichette nel dataset di training
train_dataset = dataset["train"]

# Conta le occorrenze delle etichette
label_counts = Counter(train_dataset["sentiment"])

# Stampa la distribuzione delle etichette
print("Distribuzione delle classi nel dataset di training:")
for label, count in label_counts.items():
    print(f"Classe {label}: {count} esempi")





c:\Users\nicol\anaconda3\envs\sentiment_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Distribuzione delle classi nel dataset di training:
Classe 0: 800000 esempi
Classe 4: 800000 esempi


In [24]:
dataset["train"][0]

{'text': "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D",
 'labels': 0}

In [4]:
dataset.shape

{'train': (1600000, 5), 'test': (498, 5)}

In [5]:
from transformers import AutoTokenizer
import re

# Inizializza il tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Preprocessing del testo
def preprocess_text(text):
    if not isinstance(text, str):
        return ""  # Restituisci una stringa vuota per valori non validi
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # Rimuovi URL
    text = re.sub(r"@\w+", '[USER]', text)  # Rimuovi menzioni
    text = re.sub(r"#(\w+)", r"\1", text)  # Mantieni solo la parola del hashtag
    text = re.sub(r"[!?]{2,}", '!', text)  # Riduci eccessi di punteggiatura
    text = re.sub(r"\.{2,}", '.', text)  # Riduci punti ripetuti
    text = text.lower()  # Converti in minuscolo
    text = re.sub(r"[^a-zA-Z0-9\s]", '', text)  # Rimuovi simboli non alfanumerici
    return text

# Funzione di preprocessing e tokenizzazione
def preprocess_and_tokenize(batch):
    texts = [preprocess_text(text) for text in batch["text"]]
    tokens = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=128
    )
    return {
        "input_ids": tokens["input_ids"],
        "attention_mask": tokens["attention_mask"],
        "labels": batch["labels"]  # Utilizza 'labels' per le etichette
    }

# Rinomina la colonna 'sentiment' in 'labels'
dataset = dataset.rename_column("sentiment", "labels")

# Rimuovi colonne non necessarie
dataset = dataset.remove_columns(["date", "user", "query"])  # Mantieni solo 'text' e 'labels'

# Applica preprocessing e tokenizzazione
tokenized_dataset = dataset.map(preprocess_and_tokenize, batched=True)







Map: 100%|██████████| 498/498 [00:00<00:00, 6937.59 examples/s]


In [6]:
# Controlla il risultato
tokenized_dataset["train"][0]

{'text': "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D",
 'labels': 0,
 'input_ids': [101,
  5310,
  1037,
  2008,
  2015,
  1037,
  26352,
  5017,
  2017,
  2323,
  2050,
  2288,
  2585,
  12385,
  1997,
  2353,
  2154,
  2000,
  2079,
  2009,
  1040,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,


In [7]:
# Rimuovi la colonna originale "text" dopo la tokenizzazione
tokenized_dataset = tokenized_dataset.remove_columns(["text"])

# Controlla il risultato
tokenized_dataset["train"][0]


{'labels': 0,
 'input_ids': [101,
  5310,
  1037,
  2008,
  2015,
  1037,
  26352,
  5017,
  2017,
  2323,
  2050,
  2288,
  2585,
  12385,
  1997,
  2353,
  2154,
  2000,
  2079,
  2009,
  1040,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [8]:
split_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]


In [9]:
unique_labels = set(train_dataset["labels"]) | set(test_dataset["labels"])
print(f"Etichette uniche nel dataset: {unique_labels}")

Etichette uniche nel dataset: {0, 4}


In [10]:
def map_labels(example):
    if example["labels"] == 0:
        example["labels"] = 0  # Negativo
    elif example["labels"] == 4:
        example["labels"] = 1  # Positivo
    return example

# Applica il mapping
train_dataset = train_dataset.map(map_labels)
test_dataset = test_dataset.map(map_labels)

Map: 100%|██████████| 160000/160000 [00:10<00:00, 15285.95 examples/s]


In [11]:
unique_labels = set(train_dataset["labels"]) | set(test_dataset["labels"])
print(f"Etichette uniche nel dataset: {unique_labels}")

Etichette uniche nel dataset: {0, 1}


In [52]:
import time
import numpy as np
import evaluate
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Disabilita Triton per la compatibilità con la GTX 1060
import torch._dynamo
torch._dynamo.config.suppress_errors = True

# Shuffle dei dati e selezione di un sottoinsieme
train_dataset = train_dataset.shuffle(seed=42).select(range(min(100000, len(train_dataset))))  # Usa solo 100k esempi
test_dataset = test_dataset.shuffle(seed=42).select(range(min(2000, len(test_dataset))))  # Usa solo 2k esempi

# Carica il modello pre-addestrato
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2  # Cambia a 2 per positivo/negativo
)

# Scongela solo gli ultimi 12 layer
for name, param in model.named_parameters():
    param.requires_grad = False  # Congela tutti i parametri inizialmente
for name, param in list(model.named_parameters())[-8:]:
    param.requires_grad = True  # Sblocca gli ultimi 12 layer

# Configura i Training Arguments con ottimizzazioni
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    save_strategy="steps",
    learning_rate=1e-5,
    lr_scheduler_type="linear",
    warmup_steps=1000,
    per_device_train_batch_size=64,  # Aumenta il batch size per accelerare
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,  # Riduci la frequenza di logging
    save_total_limit=2,
    eval_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    remove_unused_columns=False,
    fp16=True  # Precisione mista per accelerare il training
)

# Funzione per calcolare le metriche
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    prec = precision.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    rec = recall.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1_score = f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1_score}

# Configura il Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)







Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\nicol\anaconda3\envs\sentiment_env\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [53]:
train_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 100000
})

In [54]:

# Avvia il training
trainer.train()


  1%|          | 50/4689 [00:31<47:36,  1.62it/s]

{'loss': 0.7533, 'grad_norm': 4.619001865386963, 'learning_rate': 5.000000000000001e-07, 'epoch': 0.03}


  2%|▏         | 100/4689 [01:02<47:42,  1.60it/s]

{'loss': 0.7274, 'grad_norm': 4.110988616943359, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.06}


  3%|▎         | 150/4689 [01:34<47:27,  1.59it/s]

...

 99%|█████████▉| 4650/4689 [52:37<00:24,  1.60it/s]

{'loss': 0.5238, 'grad_norm': 1.8199288845062256, 'learning_rate': 1.057197072377338e-07, 'epoch': 2.98}


100%|██████████| 4689/4689 [53:02<00:00,  1.47it/s]

{'train_runtime': 3182.1902, 'train_samples_per_second': 94.275, 'train_steps_per_second': 1.474, 'train_loss': 0.549083533585008, 'epoch': 3.0}


TrainOutput(global_step=4689, training_loss=0.549083533585008, metrics={'train_runtime': 3182.1902, 'train_samples_per_second': 94.275, 'train_steps_per_second': 1.474, 'total_flos': 1.9733329152e+16, 'train_loss': 0.549083533585008, 'epoch': 3.0})

In [ ]:
import torch
torch.cuda.empty_cache()


In [55]:
results = trainer.evaluate()
print(results)


100%|██████████| 32/32 [00:18<00:00,  1.75it/s]

{'eval_loss': 0.4862086772918701, 'eval_accuracy': 0.7675, 'eval_precision': 0.7676207873933846, 'eval_recall': 0.7675, 'eval_f1': 0.7675279676969443, 'eval_runtime': 18.9204, 'eval_samples_per_second': 105.706, 'eval_steps_per_second': 1.691, 'epoch': 3.0}


In [42]:
sentences = [
    "I love this product! It's amazing.",
    "This is the worst experience I've ever had.",
    "It's okay, not great but not terrible."
]


In [48]:
test_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 2000
})

In [56]:
# Determina il dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Sposta il modello sul dispositivo
model = model.to(device)

# Sposta i dati sullo stesso dispositivo del modello
test_inputs = {
    "input_ids": torch.tensor(test_dataset["input_ids"]).to(device),
    "attention_mask": torch.tensor(test_dataset["attention_mask"]).to(device)
}

# Etichette reali
true_labels = torch.tensor(test_dataset["labels"]).to(device)

# Esegui le predizioni
with torch.no_grad():
    outputs = model(**test_inputs)

# Ottieni le predizioni come classi
predicted_classes = torch.argmax(outputs.logits, dim=1).cpu().numpy()

# Confronta con le etichette reali
correct_predictions = np.sum(predicted_classes == true_labels.cpu().numpy())
total_predictions = len(true_labels)

accuracy = correct_predictions / total_predictions
print(f"Accuracy sul dataset di test: {accuracy:.4f}")



Accuracy sul dataset di test: 0.7675


In [57]:
import torch

# Imposta il modello in modalità di valutazione
model.eval()

# Trasferisci il modello su GPU (se disponibile)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Frasi di test
sentences = [
    "I love this product! It's amazing.",
    "This is the worst experience I've ever had.",
    "It's okay, not great but not terrible."
]

# Tokenizza le frasi di test
inputs = tokenizer(
    sentences,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Rimuovi `token_type_ids` se esiste negli input
if "token_type_ids" in inputs:
    del inputs["token_type_ids"]

# Trasferisci gli input sulla GPU (se disponibile)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Esegui le previsioni
with torch.no_grad():
    outputs = model(**inputs)

# Calcola le probabilità e ottieni le classi predette
probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
predicted_classes = torch.argmax(probabilities, dim=-1)

# Stampa i risultati
for i, sentence in enumerate(sentences):
    print(f"Sentence: {sentence}")
    print(f"Predicted Class: {predicted_classes[i].item()}")
    print(f"Probabilities: {probabilities[i].cpu().numpy()}")
    print()





Sentence: I love this product! It's amazing.
Predicted Class: 1
Probabilities: [0.0288162  0.97118384]

Sentence: This is the worst experience I've ever had.
Predicted Class: 0
Probabilities: [0.9548583  0.04514164]

Sentence: It's okay, not great but not terrible.
Predicted Class: 0
Probabilities: [0.77398604 0.22601396]



In [58]:
model.save_pretrained("./model76")
tokenizer.save_pretrained("./model76")


('./model76\\tokenizer_config.json',
 './model76\\special_tokens_map.json',
 './model76\\vocab.txt',
 './model76\\added_tokens.json',
 './model76\\tokenizer.json')

In [48]:
import pandas as pd

# Percorso dei file scaricati
file_paths = [
    "../data/geoemotions/goemotions_1.csv",
    "../data/geoemotions/goemotions_2.csv",
    "../data/geoemotions/goemotions_3.csv",
]

# Leggi e unisci i file
dataframes = [pd.read_csv(file) for file in file_paths]
full_dataset = pd.concat(dataframes, ignore_index=True)

# Visualizza alcune righe
full_dataset.head()



,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,...,0,0,0,0,0,0,0,0,0,1


In [49]:
print(full_dataset.columns)  # Vedi le colonne
print(full_dataset.info())   # Dettagli del dataset
print(full_dataset["text"].head())  # Colonna con i testi


Index(['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211225 entries, 0 to 211224
Data columns (total 37 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   text                  211225 non-null  object 
 1   id                    211225 non-null  object 
 2   author                211225 non-null  object 
 3   subreddit             211225 non-null  object 
 4   link_id               211225 non-null  object 
 5   par

In [50]:
# Seleziona solo il testo e le emozioni
emotions_columns = [
    "admiration", "amusement", "anger", "annoyance", "approval",
    "caring", "confusion", "curiosity", "desire", "disappointment",
    "disapproval", "disgust", "embarrassment", "excitement",
    "fear", "gratitude", "grief", "joy", "love", "nervousness",
    "optimism", "pride", "realization", "relief", "remorse",
    "sadness", "surprise", "neutral"
]

filtered_dataset = full_dataset[["text"] + emotions_columns]

# Controlla il risultato
filtered_dataset.head()


,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [51]:
# Rimuovi righe con valori mancanti
filtered_dataset = filtered_dataset.dropna()

# Controlla la dimensione del dataset pulito
print(filtered_dataset.shape)


(211225, 29)


In [52]:
# Trova l'emozione dominante per ogni riga
def get_dominant_emotion(row):
    emotions = row[emotions_columns]
    if emotions.sum() == 0:  # Se non c'è alcuna emozione, considera "neutral"
        return "neutral"
    return emotions.idxmax()  # Restituisce la colonna/emozione con valore più alto

filtered_dataset["label"] = filtered_dataset.apply(get_dominant_emotion, axis=1)

# Mantieni solo testo e etichette
final_dataset = filtered_dataset[["text", "label"]]

# Controlla il risultato
final_dataset.head()


,text,label
0,That game hurt.,sadness
1,>sexuality shouldn’t be a grouping category I...,neutral
2,"You do right, if you don't care then fuck 'em!",neutral
3,Man I love reddit.,love
4,"[NAME] was nowhere near them, he was by the Fa...",neutral


In [53]:
filtered_dataset["text"] = filtered_dataset["text"].str.strip()
filtered_dataset["text"] = filtered_dataset["text"].str.lower()
import re

filtered_dataset["text"] = filtered_dataset["text"].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '', x, flags=re.MULTILINE))
filtered_dataset["text"] = filtered_dataset["text"].apply(lambda x: re.sub(r'@\w+|#\w+', '', x))
import string

filtered_dataset["text"] = filtered_dataset["text"].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
filtered_dataset["text"] = filtered_dataset["text"].apply(lambda x: re.sub(r'\d+', '', x))
from nltk.corpus import stopwords
import nltk

# Scarica le stop words (solo al primo utilizzo)
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
filtered_dataset["text"] = filtered_dataset["text"].apply(
    lambda x: ' '.join(word for word in x.split() if word not in stop_words)
)
filtered_dataset["text"] = filtered_dataset["text"].apply(lambda x: re.sub(r'(.)\1{2,}', r'\1', x))
filtered_dataset = filtered_dataset[filtered_dataset["text"].str.len() > 5]
print(filtered_dataset.head())  # Controlla i dati
print(filtered_dataset["text"].isnull().sum())  # Verifica dati mancanti


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                text  admiration  amusement  \
0                                          game hurt           0          0   
1  sexuality shouldn’t grouping category makes di...           0          0   
2                            right dont care fuck em           0          0   
3                                    man love reddit           0          0   
4                           name nowhere near falcon           0          0   

   anger  annoyance  approval  caring  confusion  curiosity  desire  ...  \
0      0          0         0       0          0          0       0  ...   
1      0          0         0       0          0          0       0  ...   
2      0          0         0       0          0          0       0  ...   
3      0          0         0       0          0          0       0  ...   
4      0          0         0       0          0          0       0  ...   

   nervousness  optimism  pride  realization  relief  remorse  sadne

In [54]:
from sklearn.model_selection import train_test_split

# Dividi il dataset in training (80%) e test (20%)
train_data, test_data = train_test_split(filtered_dataset, test_size=0.2, random_state=42)

# Dividi il test set in validation (10%) e test (10%)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

# Mostra le dimensioni dei set
print(f"Training set: {len(train_data)} esempi")
print(f"Validation set: {len(val_data)} esempi")
print(f"Test set: {len(test_data)} esempi")


Training set: 165791 esempi
Validation set: 20724 esempi
Test set: 20724 esempi


In [55]:
# Crea una mappa per le etichette testuali
label_map = {label: idx for idx, label in enumerate(filtered_dataset['label'].unique())}

# Applica la mappa al dataset
train_data["label"] = train_data["label"].map(label_map)
val_data["label"] = val_data["label"].map(label_map)
test_data["label"] = test_data["label"].map(label_map)

# Converti le etichette in tensori
train_labels = torch.tensor(list(train_data["label"].values))
val_labels = torch.tensor(list(val_data["label"].values))
test_labels = torch.tensor(list(test_data["label"].values))


In [56]:
from transformers import AutoTokenizer
import torch

# Specifica il tokenizer di RoBERTa
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# Funzione per tokenizzare il testo
def tokenize_data(data):
    return tokenizer(
        list(data["text"]),
        padding=True,
        truncation=True,
        max_length=128,  # Lunghezza massima delle sequenze
        return_tensors="pt"
    )

# Tokenizza i dati
train_encodings = tokenize_data(train_data)
val_encodings = tokenize_data(val_data)
test_encodings = tokenize_data(test_data)

# Converti le etichette in tensori
train_labels = torch.tensor(list(train_data["label"].values))
val_labels = torch.tensor(list(val_data["label"].values))
test_labels = torch.tensor(list(test_data["label"].values))

print("Tokenizzazione completata con RoBERTa!")



Tokenizzazione completata con RoBERTa!


In [57]:
from torch.utils.data import Dataset

class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item


In [58]:
train_dataset = EmotionDataset(train_encodings, train_labels)
val_dataset = EmotionDataset(val_encodings, val_labels)
test_dataset = EmotionDataset(test_encodings, test_labels)


In [65]:
# Numero di etichette (quante emozioni ci sono nel dataset)
num_labels = len(label_map)

from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=num_labels  # Numero di classi nel dataset
)



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [66]:
for name, param in model.named_parameters():
    if "classifier" not in name:  # Congela tutto tranne il classificatore
        param.requires_grad = False




In [67]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    save_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    fp16=True  # Precisione mista per accelerare il training
)




In [72]:
import numpy as np
import evaluate

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    return {"accuracy": acc, "f1": f1}


In [73]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)



In [75]:
trainer.train()


  0%|          | 6/7773 [00:05<2:02:24,  1.06it/s]
C:\Users\nicol\AppData\Local\Temp\ipykernel_41140\1098516869.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\nicol\AppData\Local\Temp\ipykernel_41140\1098516869.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx])
  1%|          | 50/7773 [00:30<1:17:35,  1.66it/s]

{'loss': 2.8868, 'grad_norm': 1.9017785787582397, 'learning_rate': 1.987134954329088e-05, 'epoch': 0.02}


                                                   
  1%|          | 50/7773 [01:49<1:17:35,  1.66it/s]C:\Users\nicol\AppData\Local\Temp\ipykernel_41140\1098516869.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\nicol\AppData\Local\Temp\ipykernel_41140\1098516869.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx])


{'eval_loss': 2.8515279293060303, 'eval_accuracy': 0.2757189731712025, 'eval_f1': 0.11918134599442098, 'eval_runtime': 78.6087, 'eval_samples_per_second': 263.635, 'eval_steps_per_second': 4.122, 'epoch': 0.02}


  1%|▏         | 100/7773 [02:19<1:17:16,  1.65it/s]

...

                                                    
  1%|▏         | 100/7773 [03:38<1:17:16,  1.65it/s]C:\Users\nicol\AppData\Local\Temp\ipykernel_41140\1098516869.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\nicol\AppData\Local\Temp\ipykernel_41140\1098516869.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx])


{'eval_loss': 2.8329453468322754, 'eval_accuracy': 0.2757189731712025, 'eval_f1': 0.11918134599442098, 'eval_runtime': 78.8683, 'eval_samples_per_second': 262.767, 'eval_steps_per_second': 4.108, 'epoch': 0.04}


  2%|▏         | 150/7773 [04:08<1:16:41,  1.66it/s] 

{'loss': 2.8375, 'grad_norm': 1.3091051578521729, 'learning_rate': 1.9614048629872636e-05, 'epoch': 0.06}


KeyboardInterrupt: 

# Geoemotions

In [1]:
import pandas as pd
import torch
import re
import random
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
from torch.utils.data import Dataset
import evaluate
import wandb

# Login a W&B (fai wandb login da terminale se non lo hai ancora fatto)
wandb.init(project="il_tuo_progetto", name="il_tuo_esperimento")

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

known_emotions_list = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity',
    'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
    'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]

file_paths = [
    '../data/geoemotions/goemotions_1.csv',
    '../data/geoemotions/goemotions_2.csv',
    '../data/geoemotions/goemotions_3.csv'
]
dataframes = [pd.read_csv(file) for file in file_paths]
combined_data = pd.concat(dataframes, ignore_index=True)

emotion_columns = [col for col in combined_data.columns if col in known_emotions_list]
if not emotion_columns:
    raise ValueError("Nessuna colonna di emozione trovata nel dataset.")

combined_data['label'] = combined_data[emotion_columns].idxmax(axis=1)

combined_data = combined_data[combined_data['text'].astype(str).str.strip().astype(bool)]
combined_data = combined_data[combined_data[emotion_columns].sum(axis=1) > 0]

unique_labels = combined_data['label'].unique()
label_mapping = {label: i for i, label in enumerate(unique_labels)}
combined_data['label'] = combined_data['label'].map(label_mapping)

def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-z0-9\s\.,!?;]', '', text)
    text = text.strip()
    return text

combined_data['cleaned_text'] = combined_data['text'].apply(preprocess_text)

class_counts = combined_data['label'].value_counts()
min_class_count = class_counts.min()
balanced_data = combined_data.groupby('label').apply(lambda x: x.sample(n=min_class_count, random_state=SEED)).reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(
    balanced_data['cleaned_text'],
    balanced_data['label'],
    test_size=0.2,
    random_state=SEED,
    stratify=balanced_data['label']
)

class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts.reset_index(drop=True)
        self.labels = labels.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts.iloc[index]
        label = self.labels.iloc[index]

        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
train_dataset = EmotionDataset(X_train, y_train, tokenizer, max_len=128)
test_dataset = EmotionDataset(X_test, y_test, tokenizer, max_len=128)




c:\Users\nicol\anaconda3\envs\sentiment_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\nicol\_netrc


C:\Users\nicol\AppData\Local\Temp\ipykernel_65984\2147234749.py:60: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_data = combined_data.groupby('label').apply(lambda x: x.sample(n=min_class_count, random_state=SEED)).reset_index(drop=True)


In [3]:
model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=len(label_mapping)
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",   # valutazione ogni N step
    eval_steps=500,                # valuta ogni 500 step
    save_strategy="steps",         # salva ogni N step
    save_steps=500,                # salva ogni 500 step
    logging_dir="./logs",
    logging_steps=10,
    seed=SEED,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    learning_rate=2e-5,
    report_to="wandb"
)


early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,
    early_stopping_threshold=0.0
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

trainer.train()
eval_results = trainer.evaluate()
print("Test results:", eval_results)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\nicol\AppData\Local\Temp\ipykernel_65984\567842942.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
  0%|          | 10/3460 [00:05<32:13,  1.78it/s]

{'loss': 3.341, 'grad_norm': 3.9538540840148926, 'learning_rate': 1.994219653179191e-05, 'epoch': 0.01}


  1%|          | 20/3460 [00:11<32:17,  1.78it/s]

{'loss': 3.3515, 'grad_norm': 2.9297754764556885, 'learning_rate': 1.9884393063583815e-05, 'epoch': 0.03}


  1%|          | 30/3460 [00:16<31:09,  1.83it/s]

...

{'eval_loss': 2.3366458415985107, 'eval_accuracy': 0.37368991687748465, 'eval_runtime': 30.2366, 'eval_samples_per_second': 91.512, 'eval_steps_per_second': 5.722, 'epoch': 3.61}


 72%|███████▏  | 2500/3460 [25:58<09:58,  1.60it/s]


{'train_runtime': 1558.4113, 'train_samples_per_second': 35.501, 'train_steps_per_second': 2.22, 'train_loss': 2.2469180938720705, 'epoch': 3.61}


100%|██████████| 173/173 [00:30<00:00,  5.74it/s]

Test results: {'eval_loss': 2.303861141204834, 'eval_accuracy': 0.37621973256234187, 'eval_runtime': 30.3332, 'eval_samples_per_second': 91.22, 'eval_steps_per_second': 5.703, 'epoch': 3.61271676300578}


In [6]:
import torch
import gc

# Svuota la cache GPU
torch.cuda.empty_cache()

# Forza la garbage collection
gc.collect()

# (opzionale) Stampa lo stato della memoria GPU
print(torch.cuda.memory_summary(device='cuda', abbreviated=True))



|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   1448 MiB |   2700 MiB |  13253 GiB |  13252 GiB |
|---------------------------------------------------------------------------|
| Active memory         |   1448 MiB |   2700 MiB |  13253 GiB |  13252 GiB |
|---------------------------------------------------------------------------|
| Requested memory      |   1442 MiB |   2690 MiB |  13178 GiB |  13177 GiB |
|---------------------------------------------------------------

In [5]:
del model
del optimizer
del train_loader
del val_loader
torch.cuda.empty_cache()


NameError: name 'optimizer' is not defined

# Prova 2

In [1]:
import pandas as pd
import torch
import re
import random
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import DebertaTokenizer, DebertaForSequenceClassification
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
from torch.utils.data import Dataset
import evaluate
import wandb

# Inizializzazione W&B (modifica 'il_tuo_progetto' e 'il_tuo_esperimento')
wandb.init(project="il_tuo_progetto", name="il_tuo_esperimento")

# Impostazione seed per riproducibilità
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

known_emotions_list = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity',
    'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
    'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]

# Carica i file CSV (adatta i nomi dei file se necessario)
file_paths = [
    '../data/geoemotions/goemotions_1.csv',
    '../data/geoemotions/goemotions_2.csv',
    '../data/geoemotions/goemotions_3.csv'
]

dataframes = [pd.read_csv(file) for file in file_paths]
combined_data = pd.concat(dataframes, ignore_index=True)

emotion_columns = [col for col in combined_data.columns if col in known_emotions_list]
if not emotion_columns:
    raise ValueError("Nessuna colonna di emozione trovata nel dataset.")

combined_data['label'] = combined_data[emotion_columns].idxmax(axis=1)
combined_data = combined_data[combined_data['text'].astype(str).str.strip().astype(bool)]
combined_data = combined_data[combined_data[emotion_columns].sum(axis=1) > 0]

unique_labels = combined_data['label'].unique()
label_mapping = {label: i for i, label in enumerate(unique_labels)}
combined_data['label'] = combined_data['label'].map(label_mapping)

def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-z0-9\s\.,!?;]', '', text)
    text = text.strip()
    return text

combined_data['cleaned_text'] = combined_data['text'].apply(preprocess_text)

# Bilanciamento per classi
class_counts = combined_data['label'].value_counts()
min_class_count = class_counts.min()
balanced_data = combined_data.groupby('label').apply(lambda x: x.sample(n=min_class_count, random_state=SEED)).reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(
    balanced_data['cleaned_text'],
    balanced_data['label'],
    test_size=0.2,
    random_state=SEED,
    stratify=balanced_data['label']
)

class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts.reset_index(drop=True)
        self.labels = labels.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts.iloc[index]
        label = self.labels.iloc[index]

        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")
train_dataset = EmotionDataset(X_train, y_train, tokenizer, max_len=128)
test_dataset = EmotionDataset(X_test, y_test, tokenizer, max_len=128)




c:\Users\nicol\anaconda3\envs\sentiment_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: nicologiuse2003 (nicologiuse2003-ironhack). Use `wandb login --relogin` to force relogin


C:\Users\nicol\AppData\Local\Temp\ipykernel_68436\1458320960.py:63: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_data = combined_data.groupby('label').apply(lambda x: x.sample(n=min_class_count, random_state=SEED)).reset_index(drop=True)


In [2]:
model = DebertaForSequenceClassification.from_pretrained(
    "microsoft/deberta-base",
    num_labels=len(label_mapping)
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Parametri di training aggiornati
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,              # più epoche per migliorare l'apprendimento
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",     # valuta alla fine di ogni epoca
    save_strategy="epoch",           # salva il modello migliore ad ogni epoca
    logging_dir="./logs",
    logging_steps=50,
    seed=SEED,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    learning_rate=2e-5,
    lr_scheduler_type="linear",      # scheduler lineare per il learning rate
    report_to="wandb",               # log su Weights & Biases
    fp16=True                        # mixed precision per training più veloce e stabile
)

# Early Stopping con più pazienza
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=5,
    early_stopping_threshold=0.0
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

trainer.train()
eval_results = trainer.evaluate()
print("Test results:", eval_results)

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\nicol\anaconda3\envs\sentiment_env\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\nicol\AppData\Local\Temp\ipykernel_68436\3798508077.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `Train

{'loss': 3.3387, 'grad_norm': 1.451082706451416, 'learning_rate': 1.985549132947977e-05, 'epoch': 0.07}


  1%|▏         | 100/6920 [01:09<1:16:51,  1.48it/s]

{'loss': 3.2845, 'grad_norm': 4.820042133331299, 'learning_rate': 1.9713872832369944e-05, 'epoch': 0.14}


  2%|▏         | 150/6920 [01:43<1:17:57,  1.45it/s]

...

{'eval_loss': 2.838918447494507, 'eval_accuracy': 0.3509215757137694, 'eval_runtime': 34.6308, 'eval_samples_per_second': 79.9, 'eval_steps_per_second': 4.996, 'epoch': 7.0}


 70%|███████   | 4844/6920 [1:00:30<25:55,  1.33it/s]


{'train_runtime': 3630.5141, 'train_samples_per_second': 30.478, 'train_steps_per_second': 1.906, 'train_loss': 1.706913396014742, 'epoch': 7.0}


100%|██████████| 173/173 [00:34<00:00,  5.00it/s]

Test results: {'eval_loss': 2.3015594482421875, 'eval_accuracy': 0.37188290567401516, 'eval_runtime': 34.8221, 'eval_samples_per_second': 79.461, 'eval_steps_per_second': 4.968, 'epoch': 7.0}


In [3]:
# Supponendo di aver già definito:
# test_dataset = EmotionDataset(X_test, y_test, tokenizer, max_len=128)

# Se hai utilizzato Trainer per l’addestramento, puoi facilmente eseguire:
test_results = trainer.evaluate(eval_dataset=test_dataset)
print("Test results:", test_results)


100%|██████████| 173/173 [00:34<00:00,  5.01it/s]

Test results: {'eval_loss': 2.3015594482421875, 'eval_accuracy': 0.37188290567401516, 'eval_runtime': 34.7901, 'eval_samples_per_second': 79.534, 'eval_steps_per_second': 4.973, 'epoch': 7.0}


In [4]:
predictions = trainer.predict(test_dataset)
preds = predictions.predictions
labels = predictions.label_ids
metrics = predictions.metrics

print("Metrics on test set:", metrics)


100%|██████████| 173/173 [00:34<00:00,  4.99it/s]

Metrics on test set: {'test_loss': 2.3015594482421875, 'test_accuracy': 0.37188290567401516, 'test_runtime': 34.8764, 'test_samples_per_second': 79.337, 'test_steps_per_second': 4.96}


In [5]:
predictions = trainer.predict(test_dataset)
print(predictions.metrics)

preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids
# Ora puoi confrontare preds e labels


100%|██████████| 173/173 [00:34<00:00,  5.00it/s]

{'test_loss': 2.3015594482421875, 'test_accuracy': 0.37188290567401516, 'test_runtime': 34.855, 'test_samples_per_second': 79.386, 'test_steps_per_second': 4.963}


In [6]:
import numpy as np

logits = predictions.predictions
preds = np.argmax(logits, axis=1)  # Indici delle classi predette
labels = predictions.label_ids      # Etichette reali


In [7]:
for i in range(10):  # primi 10 esempi
    print(f"Example {i}: Prediction = {preds[i]}, Label = {labels[i]}")


Example 0: Prediction = 16, Label = 19
Example 1: Prediction = 14, Label = 27
Example 2: Prediction = 19, Label = 27
Example 3: Prediction = 7, Label = 7
Example 4: Prediction = 20, Label = 5
Example 5: Prediction = 5, Label = 1
Example 6: Prediction = 14, Label = 14
Example 7: Prediction = 0, Label = 0
Example 8: Prediction = 21, Label = 21
Example 9: Prediction = 12, Label = 9


In [8]:
from scipy.special import expit  # funzione sigmoid

probs = expit(logits)  # se i logit sono raw
predicted_labels = [np.where(probs[i] > 0.5)[0] for i in range(len(probs))]


In [9]:
import evaluate

f1_metric = evaluate.load("f1")

# Calcolo dell'F1 macro
f1_macro = f1_metric.compute(predictions=preds, references=labels, average="macro")
print("F1 Macro:", f1_macro['f1'])

# Calcolo dell'F1 weighted
f1_weighted = f1_metric.compute(predictions=preds, references=labels, average="weighted")
print("F1 Weighted:", f1_weighted['f1'])


F1 Macro: 0.35506979467585403
F1 Weighted: 0.35521537526526514


In [2]:
import torchaudio
from torchaudio.backend.utils import get_audio_backend

# Mostra il backend audio attualmente in uso
print("Audio Backend:", get_audio_backend())

# Imposta il backend su 'ffmpeg' se disponibile
try:
    torchaudio.backend.utils.set_audio_backend("ffmpeg")
    print("FFmpeg configurato correttamente.")
except Exception as e:
    print("Errore durante la configurazione di FFmpeg:", e)



ModuleNotFoundError: No module named 'torchaudio.backend.utils'

In [11]:
import os
os.environ["PATH"] += os.pathsep + r"D:\Programmi\ffmpeg-master-latest-win64-gpl\bin"


In [12]:
import torchaudio

try:
    torchaudio.set_audio_backend("ffmpeg")
    print("Audio Backend impostato su FFmpeg.")
except Exception as e:
    print("Errore durante l'impostazione del backend FFmpeg:", e)


Audio Backend impostato su FFmpeg.


C:\Users\nicol\AppData\Local\Temp\ipykernel_51600\2085261925.py:4: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("ffmpeg")


In [13]:

file_path = "merry-christmas-to-all-to-all-a-good-night-santa-voice-david-h-m-lambert-1-1-00-06.mp3"  # Sostituisci con il percorso di un file audio
try:
    waveform, sample_rate = torchaudio.load(file_path)
    print("Caricamento audio riuscito!")
    print("Forma del waveform:", waveform.shape)
    print("Frequenza di campionamento:", sample_rate)
except Exception as e:
    print("Errore durante il caricamento del file audio:", e)


Errore durante il caricamento del file audio: Couldn't find appropriate backend to handle uri merry-christmas-to-all-to-all-a-good-night-santa-voice-david-h-m-lambert-1-1-00-06.mp3 and format None.


In [6]:
import torchaudio

# Mostra il backend audio attualmente configurato
print("Audio Backend:", torchaudio.get_audio_backend())


Audio Backend: None


C:\Users\nicol\AppData\Local\Temp\ipykernel_51600\1827802570.py:4: UserWarning: torchaudio._backend.get_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  print("Audio Backend:", torchaudio.get_audio_backend())


# Prova 3

In [1]:
import pandas as pd
import torch
import re
import random
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import (AutoTokenizer, DebertaForSequenceClassification,
                          Trainer, TrainingArguments, DataCollatorWithPadding, 
                          EarlyStoppingCallback)
from torch.utils.data import Dataset
import evaluate
import wandb

wandb.init(project="il_tuo_progetto", name="deberta_base_simplified")

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

known_emotions_list = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity',
    'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
    'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]

file_paths = [
    '../data/geoemotions/goemotions_1.csv',
    '../data/geoemotions/goemotions_2.csv',
    '../data/geoemotions/goemotions_3.csv'
]

dataframes = [pd.read_csv(file) for file in file_paths]
combined_data = pd.concat(dataframes, ignore_index=True)

emotion_columns = [col for col in combined_data.columns if col in known_emotions_list]
if not emotion_columns:
    raise ValueError("Nessuna colonna di emozione trovata nel dataset.")

combined_data['label'] = combined_data[emotion_columns].idxmax(axis=1)
combined_data = combined_data[combined_data['text'].astype(str).str.strip().astype(bool)]
combined_data = combined_data[combined_data[emotion_columns].sum(axis=1) > 0]

unique_labels = combined_data['label'].unique()
label_mapping = {label: i for i, label in enumerate(unique_labels)}
combined_data['label'] = combined_data['label'].map(label_mapping)

def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-z0-9\s\.,!?;]', '', text)
    text = text.strip()
    return text

combined_data['cleaned_text'] = combined_data['text'].apply(preprocess_text)

class_counts = combined_data['label'].value_counts()
min_class_count = class_counts.min()
balanced_data = combined_data.groupby('label').apply(lambda x: x.sample(n=min_class_count, random_state=SEED)).reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(
    balanced_data['cleaned_text'],
    balanced_data['label'],
    test_size=0.2,
    random_state=SEED,
    stratify=balanced_data['label']
)

class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts.reset_index(drop=True)
        self.labels = labels.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts.iloc[index]
        label = self.labels.iloc[index]

        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
train_dataset = EmotionDataset(X_train, y_train, tokenizer, max_len=128)
test_dataset = EmotionDataset(X_test, y_test, tokenizer, max_len=128)

num_labels = len(label_mapping)
class_counts = y_train.value_counts()
total = len(y_train)
class_weights = [total/class_counts[i] for i in range(len(class_counts))]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

model = DebertaForSequenceClassification.from_pretrained(
    "microsoft/deberta-base",
    num_labels=num_labels
).to(device)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_metric.compute(predictions=preds, references=labels)
    f1_macro = f1_metric.compute(predictions=preds, references=labels, average="macro")
    f1_weighted = f1_metric.compute(predictions=preds, references=labels, average="weighted")
    return {
        "accuracy": acc["accuracy"],
        "f1_macro": f1_macro["f1"],
        "f1_weighted": f1_weighted["f1"]
    }

from transformers.trainer import Trainer

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**{k: v for k, v in inputs.items() if k != "labels"})
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    seed=SEED,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    learning_rate=2e-5,
    lr_scheduler_type="linear",
    report_to="wandb",
    fp16=True
)

early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.0
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

trainer.train()
eval_results = trainer.evaluate()
print("Test results:", eval_results)


c:\Users\nicol\anaconda3\envs\sentiment_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: nicologiuse2003 (nicologiuse2003-ironhack). Use `wandb login --relogin` to force relogin


C:\Users\nicol\AppData\Local\Temp\ipykernel_79704\2366285596.py:60: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_data = combined_data.groupby('label').apply(lambda x: x.sample(n=min_class_count, random_state=SEED)).reset_index(drop=True)
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\nicol\anaconda3\envs\sentiment_env\lib\site-packages\transformers\training_args.py:1568: FutureWarning:

{'loss': 3.3387, 'grad_norm': 1.450717568397522, 'learning_rate': 1.971098265895954e-05, 'epoch': 0.07}


  3%|▎         | 100/3460 [01:10<38:10,  1.47it/s]

...

{'eval_loss': 2.377781391143799, 'eval_accuracy': 0.36393205637874954, 'eval_f1_macro': 0.35319960084597957, 'eval_f1_weighted': 0.3533029138940354, 'eval_runtime': 34.7328, 'eval_samples_per_second': 79.665, 'eval_steps_per_second': 4.981, 'epoch': 5.0}


100%|██████████| 3460/3460 [42:53<00:00,  1.34it/s]


{'train_runtime': 2573.5769, 'train_samples_per_second': 21.497, 'train_steps_per_second': 1.344, 'train_loss': 2.127726753874321, 'epoch': 5.0}


100%|██████████| 173/173 [00:34<00:00,  5.06it/s]

Test results: {'eval_loss': 2.377781391143799, 'eval_accuracy': 0.36393205637874954, 'eval_f1_macro': 0.35319960084597957, 'eval_f1_weighted': 0.3533029138940354, 'eval_runtime': 34.3816, 'eval_samples_per_second': 80.479, 'eval_steps_per_second': 5.032, 'epoch': 5.0}


In [213]:
from collections import Counter

# Conta le occorrenze delle etichette nel dataset di training
train_label_counts = Counter([int(label) for label in train_dataset['labels']])
test_label_counts = Counter([int(label) for label in test_dataset['labels']])

# Stampa la distribuzione in modo leggibile
print("Distribuzione delle classi nel dataset di training:")
for label, count in train_label_counts.items():
    print(f"Classe {label}: {count} esempi")

print("\nDistribuzione delle classi nel dataset di test:")
for label, count in test_label_counts.items():
    print(f"Classe {label}: {count} esempi")



Distribuzione delle classi nel dataset di training:
Classe 0: 25063 esempi
Classe 2: 24937 esempi

Distribuzione delle classi nel dataset di test:
Classe 1: 139 esempi
Classe 0: 177 esempi
Classe 2: 182 esempi


In [ ]:
from transformers import Trainer, TrainingArguments

# Configura i parametri di training
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50
)

# Inizializza il Trainer
trainer = Trainer(
    model=model,  # Assicurati di aver caricato il modello DistilBERT
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

# Avvia il fine-tuning
trainer.train()


In [59]:
# Imposta il formato del dataset
tokenized_datasets = tokenized_datasets.remove_columns(["text", "date", "user", "query"])
tokenized_datasets.set_format("torch")

# Dividi in train e test
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]


ValueError: Column name ['date', 'user', 'query', 'text'] not in the dataset. Current columns in the dataset: ['sentiment', 'input_ids', 'attention_mask', 'labels']

In [106]:
from transformers import DistilBertForSequenceClassification

# Inizializza il modello
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [107]:
# Congela tutti i layer
for param in model.base_model.parameters():
    param.requires_grad = False

for param in model.distilbert.transformer.layer[-8:].parameters():
    param.requires_grad = True





In [108]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, Trainer, TrainingArguments
import torch.nn as nn

# Modifica il classificatore finale (opzionale)
model.classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, 128),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(128, 3)
)

In [111]:
from collections import Counter

# Mescola i dataset
train_dataset = train_dataset.shuffle(seed=42)
test_dataset = test_dataset.shuffle(seed=42)

# Seleziona fino a 50.000 esempi nel training e 2.000 nel test
small_train_dataset = train_dataset.select(range(min(50000, len(train_dataset))))
small_test_dataset = test_dataset.select(range(min(2000, len(test_dataset))))

# Configura il Trainer
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    save_strategy="steps",
    learning_rate=2e-5,
    lr_scheduler_type="linear",
    warmup_steps=500,  # Una percentuale del totale dei passi di training
    per_device_train_batch_size=32,  # Batch size aumentato
    per_device_eval_batch_size=32,
    num_train_epochs=2,  # Ridotto per test rapidi
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    eval_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True
)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    compute_metrics=compute_metrics
)

In [112]:
# Avvia l'addestramento
trainer.train()


  2%|▏         | 50/3126 [00:25<25:41,  2.00it/s]

{'loss': 1.0984, 'grad_norm': 1.2728896141052246, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.03}


  3%|▎         | 100/3126 [00:50<25:13,  2.00it/s]

{'loss': 1.0007, 'grad_norm': 1.5675368309020996, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.06}


  5%|▍         | 150/3126 [01:15<24:40,  2.01it/s]

...

{'eval_loss': 2.2044501304626465, 'eval_accuracy': 0.5963855421686747, 'eval_precision': 0.46425460879663233, 'eval_recall': 0.5963855421686747, 'eval_f1': 0.5081957533711664, 'eval_runtime': 3.0498, 'eval_samples_per_second': 163.291, 'eval_steps_per_second': 5.246, 'epoch': 0.96}


 50%|████▉     | 1550/3126 [13:11<13:32,  1.94it/s]

{'loss': 0.3876, 'grad_norm': 5.323203086853027, 'learning_rate': 1.2003046458492005e-05, 'epoch': 0.99}


 50%|████▉     | 1559/3126 [13:16<13:27,  1.94it/s]

KeyboardInterrupt: 

In [101]:
# Valutazione sul dataset di test
metrics = trainer.evaluate(eval_dataset=small_test_dataset)
print(metrics)


100%|██████████| 8/8 [00:03<00:00,  2.36it/s]

{'eval_loss': 2.4108541011810303, 'eval_runtime': 3.3459, 'eval_samples_per_second': 148.84, 'eval_steps_per_second': 2.391, 'epoch': 2.0}


In [102]:
# Previsioni sul dataset di test
predictions = trainer.predict(test_dataset=small_test_dataset)

# Estrai logits e etichette predette
logits = predictions.predictions
predicted_labels = logits.argmax(axis=1)  # Classe con probabilità massima

# Etichette reali
true_labels = predictions.label_ids

# Calcola l'accuratezza manualmente (opzionale)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuratezza: {accuracy * 100:.2f}%")


100%|██████████| 8/8 [00:02<00:00,  2.74it/s]

Accuratezza: 60.04%
